In [7]:
######## GNSS instavel: Filter and write to new phase_vel files ##################
####################################### Goals ################################################
#     1) Read a GNSS instavel and convert it into an obspy stream
#     2) Do this for all three components iteratively
#     3) re-write as phase_vel file with new file ID
##############################################################################################


from obspy import UTCDateTime 				
from obspy import Trace
from obspy import Stream
from obspy import read_inventory
from obspy.signal.cross_correlation import correlate
from obspy.clients.fdsn import Client 		
client=Client("IRIS")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as colors
import matplotlib.cm as cmx
import datetime

# LOOP OVER STATIONS ANF CREAT CHANNEL STREAMS
#st = Stream(Trace())
for sta in ["ab02", "ab07", "ab13", "ab14", "ac02", "ac08", "ac12", "ac13", "ac21", "ac25", "ac26", "ac27", 
            "ac28", "ac34", "ac39", "ac40", "ac41", "ac42", "ac45", "ac47", "ac52", "ac67"]:
    infile = f"{sta}_1hz.L2.igr.phase_vels"
    site = np.genfromtxt(infile, dtype=None, encoding=None, names=['date','time','dt','lat','lon','height','vx',
                                                                   'vy','vz','sigx','sigy','sigz','vdt',
                                                                   'vn','ve','vu','sign','sige','sigu'])

    # CREATE EMPTY STREAMs AND POPULATE WITH INSTAVEL DATA1
    st_gnss_vn = Trace()
    dt = UTCDateTime("2021-07-29T06:15:47.000000Z")
    st_gnss_vn.stats.network = '--'
    st_gnss_vn.stats.station = f'{sta}'
    st_gnss_vn.stats.channel = 'IVN'    
    st_gnss_vn.stats.starttime = site["date"][0] + 'T' + site["time"][0]
    st_gnss_vn.stats.sampling_rate = 1
    st_gnss_vn.stats.calib = 1
    st_gnss_vn.data = site["vn"]
    st_gnss_vn.detrend('linear')
    st_gnss_vn.detrend('demean')
    st_gnss_vn.trim(dt+60, dt+360)
    st_gnss_vn.filter('bandpass', freqmin=0.001, freqmax=0.25, corners=4, zerophase=True)
    st_gnss_ve = Trace()
    st_gnss_ve.stats.network = '--'
    st_gnss_ve.stats.station = f'{sta}'
    st_gnss_ve.stats.channel = 'IVE'    
    st_gnss_ve.stats.starttime = site["date"][0] + 'T' + site["time"][0]
    st_gnss_ve.stats.sampling_rate = 1
    st_gnss_ve.stats.calib = 1
    st_gnss_ve.data = site["ve"]
    st_gnss_ve.detrend('linear')
    st_gnss_ve.detrend('demean')
    st_gnss_ve.trim(dt+60, dt+360)
    st_gnss_ve.filter('bandpass', freqmin=0.001, freqmax=0.25, corners=4, zerophase=True)
    st_gnss_vz = Trace()
    st_gnss_vz.stats.network = '--'
    st_gnss_vz.stats.station = f'{sta}'
    st_gnss_vz.stats.channel = 'IVZ'    
    st_gnss_vz.stats.starttime = site["date"][0] + 'T' + site["time"][0]
    st_gnss_vz.stats.sampling_rate = 1
    st_gnss_vz.stats.calib = 1
    st_gnss_vz.data = site["vz"]
    st_gnss_vz.detrend('linear')
    st_gnss_vz.detrend('demean')
    st_gnss_vz.trim(dt+60, dt+360)
    st_gnss_vz.filter('bandpass', freqmin=0.001, freqmax=0.25, corners=4, zerophase=True)
    st_gnss_v = Stream(traces=[st_gnss_ve, st_gnss_vn, st_gnss_vz])
    print(st_gnss_v)
    date = st_gnss_ve.times()-('T' + site["time"][0])
    time = st_gnss_ve.times()-(site["date"][0] + 'T')
    delta = st_sm_ve.times() - dt
    lat = site["lat"]
    lon = site["lon"]
    H = site["height"]
    zero = 0
    pd.concat([pd.DataFrame(date), pd.DataFrame(time), pd.DataFrame(st_gnss_vn.data), 
               pd.DataFrame(zero), pd.DataFrame(zero), pd.DataFrame(zero), pd.DataFrame(zero), 
               pd.DataFrame(zero), pd.DataFrame(zero), pd.DataFrame(zero), 
               pd.DataFrame(st_gnss_ve.data), pd.DataFrame(st_gnss_vz.data), 
               pd.DataFrame(zero), pd.DataFrame(zero), pd.DataFrame(zero),], axis=1).to_csv(f'{sta}.csv')
    #np.savetxt("%s.txt" % '{sta}', dtype=None, encoding=None, np.vstack((site['date'], site['time'],
    #                                                                     site['dt'], site['lat'], 
     #                                                                    site['lon'], site['height'], 
      #                                                                   '0', '0', 
       #                                                                  '0', '0', 
        #                                                                 '0', '0', 
         #                                                                '0', st[0].data, 
          #                                                               st[1].data, st[2].data, 
           #                                                              site['sign'], site['sige'], 
            #                                                             site['sigu'])).T, delimiter="\t")

#print(st)
#print(st[1])
#print(st[2])



OSError: ab02_1hz.L2.igr.phase_vels not found.